# ETL pipeline for sample bike data
 build dictionary of counts at anygiven time

## Imports

#testing:  138, 23, 236

In [105]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np

startTime = datetime.now()

### Connect to data

In [131]:
# connect to occupancy data
occupancy = pd.read_csv('data_clean/occupancy.csv', header=0, index_col='arrival_time', parse_dates=True)

In [132]:
#occupancy.head()

In [111]:
# testing set
station_id = 236
occupancy = occupancy[occupancy.station_id == station_id]

In [112]:
#occupancy.head(2)

> 01 build an empty dictionary of date times

In [113]:
# create a range of all possible timestamps
min_dt = occupancy.index.min()
max_dt = max(occupancy.index + pd.to_timedelta(occupancy.duration, unit='m'))
dt_range = int((max_dt - min_dt).total_seconds() / 60) + 1

# create a list of all possible timestamps
dt_list = [min_dt + timedelta(minutes=i) for i in range(dt_range)]
zeros = [0] * dt_range

# turn that the list of all possible timestamps into an empty dictionary
occupancy_counts = dict(zip(dt_list,zeros))

# delete list of dates
del dt_list, zeros

> 02 populate the empty dictionary with bike counts

In [114]:
# dictionary of counts per minute at the given station
for index, row in occupancy.iterrows():
    for i in range(row['duration']):
       master_time = index + timedelta(minutes=i)
       occupancy_counts[master_time] += 1

del occupancy

### Aggregate data to 15/30 min periods

In [115]:
# convert dictionary to df
occupancy_minute = pd.DataFrame(occupancy_counts.items(), columns=['date_time','count'])

# delte dictionary
del occupancy_counts

In [116]:
# set index
occupancy_minute = occupancy_minute.set_index('date_time')

In [117]:
# import dockcount data
dpcount = pd.read_csv('data_clean/all_stations.csv', header=0)
dpcount = dpcount[dpcount.Id == station_id].reset_index(drop=True)
#dpcount['Start'] = pd.Series([datetime.strptime(i, "%m/%d/%y") for i in dpcount.Start.tolist()])
#dpcount['End'] = pd.Series([datetime.strptime(i, "%m/%d/%y") for i in dpcount.End.tolist()])

In [118]:
#test = dpcount.groupby(['Dpcapacity']).agg({'Start':'min', 'End':'max'})
#occupancy_minute[occupancy_minute.index <= test.End.min()]


In [119]:
#docklist = []
#for i in occupancy_minute.date_time.tolist():
#    for j in dpcount.iterrows():
#        if (i >= j[1][2] and i <= j[1][3]):
#            docklist.append(j[1][1])

In [120]:
dock_count = dpcount.Dpcapacity.min()

In [121]:
# flag full and empty mintues
occupancy_minute['empty'] = occupancy_minute['count'] == 0
occupancy_minute['full'] = occupancy_minute['count'] >= dock_count

In [122]:
# round mintues to 30 min (round down ie 6:59 goes to 6:30)
# group by 30 min periods
occupancy_minute = occupancy_minute.resample('30T').agg({'count':['mean','min','max'], 'empty':'sum', 'full':'sum'})

In [125]:
# rename the columns
occupancy_minute.columns = [' '.join(col).strip() for col in occupancy_minute.columns.values]

In [127]:
# rename columns
columns = ['mean_count','min_count','max_count', 'minutes_full', 'minutes_empty']
occupancy_minute.columns = columns

In [130]:
# write to csv
output_file = str(station_id) + '.csv'
occupancy_minute.to_csv('data_output/' + output_file)

print datetime.now() - startTime

0:06:39.322035
